## Noun Phrase Chunking of IMDB Movie Reviews
### DS 7337 - Natural Laguage Processing - Homework 5
#### George C. Sturrock
##### March 16, 2019
##### Homework 6 Objectives
1.  Evaluate text similarity of Amazon book search results by doing the following:

a.  Do a book search on Amazon. Manually copy the full book title (including subtitle) of each of the top 24 books listed in the first two pages of search results. 

b.  In Python, run one of the text-similarity measures covered in this course, e.g., cosine similarity. Compare each of the book titles, pairwise, to every other one. 

c.  Which two titles are the most similar to each other? Which are the most dissimilar? Where do they rank, among the first 24 results?


2.  Now evaluate using a major search engine.

a.  Enter one of the book titles from question 1a into Google, Bing, or Yahoo!. Copy the capsule of the first organic result and the 20th organic result. Take web results only (i.e., not video results), and skip sponsored results. 

b.  Run the same text similarity calculation that you used for question 1b on each of these capsules in comparison to the original query (book title). 

c.  Which one has the highest similarity measure? 

Submit all of your inputs and outputs and your code for this assignment, along with a brief written explanation of your findings. 


#### Import Libraries

In [1]:
import platform; print("Platform:", platform.platform())
import os
import sys; print("Python:", sys.version)
import requests; print("Requests:", requests.__version__)
from urllib import request; ("urllib:", request.__version__)
import warnings
warnings.filterwarnings('ignore')
from time import sleep

#Web Scraping
import bs4; print("BeatifulSoup:", bs4.__version__)
from bs4 import BeautifulSoup
import json; print("json:", json.__version__)

#Python Basics
import numpy as np; print("Numpy Version:", np.__version__)
import pandas as pd; print("Pandas Version:", pd.__version__)
import seaborn as sns;

import pattern; print("Pattern Version:", pattern.__version__)
from pattern.en import tag
from pattern.en import parsetree
from pattern.vector import Document, Model, TFIDF, TF

Platform: Windows-10-10.0.14393-SP0
Python: 3.6.8 |Anaconda, Inc.| (default, Feb 21 2019, 18:30:04) [MSC v.1916 64 bit (AMD64)]
Requests: 2.21.0
BeatifulSoup: 4.6.0
json: 2.0.9
Numpy Version: 1.15.4
Pandas Version: 0.23.0
Pattern Version: 3.6


### Question 6.1
#### Question 6.1.a
Despite Stephen Ambrose's propensity for plagiarism, "Band of Brothers" is one of my all-time favorite books.  An Amazon book search for this title returns related results for this title.  The results include several formats of the original book included paper back, Kindle, DVD, etc.  These redundant results were not included in the list below which includes the top twenty-eight related results (not including the aforementioned duplicates).  Additionally, only book results were included in the list.  T-shirts appeared in the search results even though the Amazon book search was filtered for "Books".  

In [3]:
try:
    del mb
except Exception as e:
    print(e, "-- Continue")

boblist = ["Band of Brothers: E Company, 506th Regiment, 101st Airborne from Normandy to Hitler's Eagle's Nest", 
          "Beyond Band of Brothers: The War Memoirs of Major Dick Winters", 
          "Brothers in Battle, Best of Friends: Two WWII Paratroopers from the Original Band of Brothers Tell Their Story",
          "Shifty's War: The Authorized Biography of Sergeant Darrell Powers, the Legendary Sharpshooter from the Band", 
          "Conversations with Major Dick Winters: Life Lessons from the Commander of the Band of Brothers", 
          "Easy Company Soldier: The Legendary Battles of a Sergeant from WW II's 'Band of Brothers'",
          "How Easy Company Became a Band of Brothers",
          "Biggest Brother: The Life Of Major Dick Winters, The Man Who Led The Band of Brothers", 
          "We Who Are Alive and Remain: Untold Stories from the Band of Brothers", 
          "The General's Driver: A Vietnam Soldier Remembers", 
          "Killing the SS: The Hunt for the Worst War Criminals in History (Bill O'Reilly's Killing Series)", 
          "The Allman Brothers Band Classic Memorabilia, 1969-76 (Music and the American South)", 
          "Building Your Band of Brothers", 
          "Killer Genius: The Bizarre Case of the Homicidal Scholar (Dead True Crime Book 5)", 
          "Murderer's Gulch: Carnage in the Catskills (Dead True Crime Book 4)", 
          "Band of Brothers: Battle Manual", 
          "Call of Duty: My Life Before, During and After the Band of Brothers", 
          "Band of Brothers: The Complete Campaigns", 
          "My Kid Brother's Band... a.k.a. The Beatles", 
          "One More Moment (Shaughnessy Brothers: Band on the Run Book 3)", 
          "One Way Out: The Inside History of the Allman Brothers Band", 
          "Tangled Up in You (The Shaughnessy Brothers Book 7)", 
          "The Allman Brothers Band: The Definitive Collection for Guitar, Vol. 3", 
          "The Allman Brothers Band - The Definitive Collection for Guitar - Volume 1 (Guitar Recorded Versions S)", 
          "Bastogne Band of Brothers Guide", 
          "Daisy Jones & The Six: A Novel", 
          "Band of Brothers: E Company 506th Regiment 101st Airborne, D-day & Citizen Soldiers Box Set", 
          "D DAY Through German Eyes - The Hidden Story of June 6th 1944"]

boblist2 = []

for b in range(len(boblist)):
    boblist2.append(Document(boblist[b], type=boblist[b].strip('"')))

mb = Model(documents=boblist2, weight=TFIDF)

#### Question 6.1.b
Each book title is tranlated into a Pattern Document.  The Pattern Documents are then fed into a Pattern Model to produce the comprehensive corpus of book titles related to the "Band of Brothers" Amazon book search.  Below, the Pattern Model similarity function is used to produce similarity scores for each pairwise combination of titles.  The pairwise comibination of titles and their similarity scores are written to the dfQ1 dataframe for ease of reporting.

In [4]:
dfQ1 = pd.DataFrame(columns=['Title1', 'Title2', 'CosineSim'])

cosSimScores = [(round(mb.similarity(x, y), 4), mb.documents[i].type, mb.documents[j].type) 
                for i,x in enumerate(mb.documents) 
                for j,y in enumerate(mb.documents) if i != j]

for c in cosSimScores:
#    if c[0] > 0:dfQ1 = dfQ1.append({'Title1':c[1], 'Title2':c[2], 'CosineSim':c[0]}, ignore_index=True)
#        dfQ1 = dfQ1.append({'Title1':c[1], 'Title2':c[2], 'CosineSim':c[0]}, ignore_index=True)
    dfQ1 = dfQ1.append({'Title1':c[1], 'Title2':c[2], 'CosineSim':c[0]}, ignore_index=True)

In order to remove duplicate pairwise values, the dfQ1 dataframe is sorted descending by "CosineSim" and every other record is removed.  This leaves distinct combinations of pairwise titles and cosine similarity scores.  All scores are calculated using a TFIDF weighting to highlight the truly unique tokens in the corpus.  

In [5]:
dfQ1.sort_values(by=['CosineSim'], ascending=False, inplace=True)
dfQ1 = dfQ1.iloc[::2]

In [6]:
dfQ1

,Title1,Title2,CosineSim
616,The Allman Brothers Band: The Definitive Colle...,The Allman Brothers Band - The Definitive Coll...,0.4828
25,"Band of Brothers: E Company, 506th Regiment, 1...",Band of Brothers: E Company 506th Regiment 101...,0.4663
167,How Easy Company Became a Band of Brothers,Easy Company Soldier: The Legendary Battles of...,0.4099
114,Conversations with Major Dick Winters: Life Le...,Biggest Brother: The Life Of Major Dick Winter...,0.3924
190,Biggest Brother: The Life Of Major Dick Winter...,Beyond Band of Brothers: The War Memoirs of Ma...,0.3862
30,Beyond Band of Brothers: The War Memoirs of Ma...,Conversations with Major Dick Winters: Life Le...,0.3709
533,One More Moment (Shaughnessy Brothers: Band on...,Tangled Up in You (The Shaughnessy Brothers Bo...,0.2983
391,Murderer's Gulch: Carnage in the Catskills (De...,Killer Genius: The Bizarre Case of the Homicid...,0.2884
407,Band of Brothers: Battle Manual,"Brothers in Battle, Best of Friends: Two WWII ...",0.2035
289,Killing the SS: The Hunt for the Worst War Cri...,One Way Out: The Inside History of the Allman ...,0.1898


#### Question 6.1.c
The two most similar pairs of titles are shown below.  Of course, there would have been some 100% similarity scores if various physical and electronic forms of Ambrose's book would not have been filtered from the list.  That said, the pair of titles with the highest TFIDF weighted similarity scores are books about the Allman Brothers Band.  These have little contextual similarity to World War II experiences of the paratroopers of Easy Company.  These two titles were the 23rd and 24th books returned by the Amazon search.

The second pair of titles are relevant to the topic.  They are both Ambrose WWII books, and they are similar in title.  Interestingly, these were the 1st and 27th titles returned by Amazon.  This may show Amazon uses a scoring weight other than just TFIDF to find the most relevant titles for book searches.

In [7]:
dfQ1[0:2]

,Title1,Title2,CosineSim
616,The Allman Brothers Band: The Definitive Colle...,The Allman Brothers Band - The Definitive Coll...,0.4828
25,"Band of Brothers: E Company, 506th Regiment, 1...",Band of Brothers: E Company 506th Regiment 101...,0.4663


Two of the most dissimilar results are shown below.  However, the were more than two cosine similarity scores equal to zero.  "Killer Genius: The Bizarre Case of the Homicidal Scholar (Dead True Crime Book 5)" has a 0 cosine similarity score for all pairwise comparisons.  This title was the 14th title returned by the Amazon book search.  Visual examination of titles confirms the TFIDF weighted cosine similarity score of zero for "Killer Genius: The Bizarre Case of the Homicidal Scholar (Dead True Crime Book 5)".  Additionally, it has no contextual relavance to World War II books.  Again, this suggests Amazon uses more than TFIDF weighted cosine similarity to return search results.  There could be a relation between customers who purchase books about The Band of Brothers and this title.

In [8]:
dfQ1[-2:]

,Title1,Title2,CosineSim
361,Killer Genius: The Bizarre Case of the Homicid...,Killing the SS: The Hunt for the Worst War Cri...,0.0
359,Killer Genius: The Bizarre Case of the Homicid...,We Who Are Alive and Remain: Untold Stories fr...,0.0


#### Question 6.2.a
Pattern is used again on this question to assess the similarity between a search string and search results using the Google search engine.  The search string is the title of Stephen Ambrose's book "Band of Brothers: E Company 506th Regiment, 101st Airborne from Normandy to Hitler's Eagle's Nest".  This question asks for a comparison of the search string to the full result capsules of the 1st and 20th capsules returned by Google.  As there are only three strings being compared, these values were manually copied from Google and translated to Pattern Documents and a Model.

In [9]:
#https://www.clips.uantwerpen.be/pages/pattern-vector

try:
    del m2, m2a
except Exception as e:
    print(e, "-- Continue")
    
p = ':!?()[]{}\'`"@#$*+-|=~_'

q = Document("Band of Brothers: E Company 506th Regiment, 101st Airborne from Normandy to Hitler's Eagle's Nest", type='query', name='query', punctuation=p)
c1 = Document("Band of Brothers: E Company, 506th Regiment, 101st Airborne from ... https://www.amazon.com/Band-Brothers-Regiment-Airborne-Normandy/.../07432245... Stephen E. Ambrose’s iconic New York Times bestseller about the ordinary men who became the World War II’s most extraordinary soldiers: Easy Company, 506th Parachute Infantry Regiment, 101st Airborne Division, US Army. ... Stephen E. Ambrose was a renowned historian and acclaimed", type='capsule', name='capsule1', punctuation=p)
c2 = Document("Band of brothers : E Company, 506th Regiment, 101st Airborn…https://searchworks.stanford.edu/view/4649634 Band of brothers : E Company, 506th Regiment, 101st Airborne from Normandy to Hitlers Eagles Nest. Responsibility: Stephen E. Ambrose. Edition: 2nd ...", type='capsule', name='capsule2', punctuation=p)

m2 = Model(documents=[q, c1, c2], weight=TFIDF)

name 'm2' is not defined -- Continue


#### Question 6.2.b
The TFIDF weighted similarity scores for the search query compared to the 1st capsule and 20th capsule returned in shown below.  Interestingly, the TFIDF weighted cosine similarity score for the first returned capsule and its query is zero.  The score for the 20th capsule is 0.0292.  

These relatively low scores illustrate the inner workings of TFIDF weighting.  Visual inspection of the search query, 1st and 20th capsules would suggest the capsules are highly similar.  However, TFIDF minimizes the importance of tokens appearing frequently in individual documents in the corpus.  This essentially makes "Band of Brothers: E Company 506th Regiment, 101st Airborne" unimportant when determining similarity as they appear in each document.  

In [17]:
print(" 1st Capsule:", round(m2.similarity(q, c1), 4))
print("20th Capsule:", round(m2.similarity(q, c2), 4))

 1st Capsule: 0.0
20th Capsule: 0.0292


#### Question 6.2.c
Using TFIDF weighted cosine similarity scores, the 20th result returned by Google is actually more similar than the first result.  As this is a well researched and popular book, the first - 20th results returned by Google were all very similar.  

### Extra Work - Comparing TF to TFIDF
The TFIDF weighted cosine similarity score of 0 for the first result returned by Google is surprising but not incorrect as shown above.  However, this does raise the question of how term frequency (TF) weighted cosine similarity scores would treat the 1st and 20th capsules returned by Google.  In contract to the TFIDF weighing, TF does not minimized the importance of tokens appearing in all documents in the corpus.  Regardless, TF still scores the 20th capsule as being more similar to the search string than the 1st.  Again, this topic is popular and well documented.  All of the top 20 capsules identified by Google were extremely similiar.

In [15]:
qb = Document("Band of Brothers: E Company 506th Regiment, 101st Airborne from Normandy to Hitler's Eagle's Nest", type='query', name='query', punctuation=p)
c1b = Document("Band of Brothers: E Company, 506th Regiment, 101st Airborne from ... https://www.amazon.com/Band-Brothers-Regiment-Airborne-Normandy/.../07432245... Stephen E. Ambrose’s iconic New York Times bestseller about the ordinary men who became the World War II’s most extraordinary soldiers: Easy Company, 506th Parachute Infantry Regiment, 101st Airborne Division, US Army. ... Stephen E. Ambrose was a renowned historian and acclaimed", type='capsule', name='capsule1', punctuation=p)
c2b = Document("Band of brothers : E Company, 506th Regiment, 101st Airborn…https://searchworks.stanford.edu/view/4649634 Band of brothers : E Company, 506th Regiment, 101st Airborne from Normandy to Hitlers Eagles Nest. Responsibility: Stephen E. Ambrose. Edition: 2nd ...", type='capsule', name='capsule2', punctuation=p)

m2a = Model(documents=[qb, c1b, c2b], weight=TF)
print(" 1st Capsule:", round(m2a.similarity(qb, c1b), 4))
print("20th Capsule:", round(m2a.similarity(qb, c2b), 4))

 1st Capsule: 0.4587
20th Capsule: 0.6838
